# Optimizing Display Calculations

This is a notebook containing the measurements to show the differences between two different methods of calculating the light efficiency and refresh rate of an led panel. This calculation is necessary becuase it allows the designer of the sytem that the led panel will be used in to optimize the tradoffs between the color depth, refresh rate, and light ouptut of the led panel used

In [1]:
import pandas as pd
import numpy as np

# Testing Speed of PanelConfig Class

Below is the code for the PanelConfig class which was written by Sylvain Munaut as part of the no2hub75 LED panel driver written for the Icebreaker FPGA Dev Board which was created by Esden. Checkout the no2hub repository here -> https://github.com/no2fpga/no2hub75

In [2]:
OVERHEAD = 5	# Guesstimated

class PanelConfig(object):

	def __init__(self, **kwargs):
		params = [
			'freq',				# Clock frequency in Hz
			'n_banks',			# Number of banks
			'n_rows',			# Number of rows
			'n_cols',			# Number of columns
			'n_planes',			# Number of bitplanes in BCM modulation
			'bcm_lsb_len',		# Duration of the LSB of BCM modulation (in clk cycles)
		]

		for x in params:
			setattr(self, x, kwargs.pop(x))

		self._sim()

	def _sim(self):
		# Init
		cyc_tot = 0
		cyc_on  = 0

		# Scan all place
		for plane in range(self.n_planes):
			# Length of the plane in clock cycle
			len_show = (self.bcm_lsb_len * 2**plane) #/ 2

			# Length required to do data shift for the next plane
			len_shift = self.n_cols

			# Length of this cycle is the max
			len_plane = max(len_show, len_shift) + OVERHEAD

			# Accumulate
			cyc_tot += len_plane
			cyc_on  += len_show

		# Compute results
		self._light_efficiency = 1.0 * cyc_on / cyc_tot
		self._refresh_rate = self.freq / (self.n_rows * cyc_tot)

	@property
	def light_efficiency(self):
		return self._light_efficiency

	@property
	def refresh_rate(self):
		return self._refresh_rate

Below are constants that will be used to define the parameters used in the calcualtions

In [3]:
freq_min = 25000000
freq_max = 25000000

n_banks_min = 2
n_banks_max = 2

n_rows_min = 32
n_rows_max = 32

n_cols_min = 32
n_cols_max = 32

n_planes_min = 8
n_planes_max = 8

bcm_lsb_len_min = 0
bcm_lsb_len_max = 20

### Using lists to store results

In [4]:
%%timeit
results = []

# print(results)
for freq in range(freq_min, freq_max + 1):
  for n_banks in range(n_banks_min, n_banks_max + 1):
    for n_rows in range(n_rows_min, n_rows_max + 1):
      for n_cols in range(n_cols_min, n_cols_max + 1):
        for n_planes in range(n_planes_min, n_planes_max + 1):
          for bcm_lsb_len in range(bcm_lsb_len_min, bcm_lsb_len_max + 1):
            pc = PanelConfig(
                freq = freq,
                n_banks = n_banks,
                n_rows = n_rows,
                n_cols = n_cols,
                n_planes = n_planes,
                bcm_lsb_len = bcm_lsb_len
            )
            results.append([freq, n_banks, n_rows, n_cols, n_planes, bcm_lsb_len, pc.light_efficiency * 100, pc.refresh_rate])

153 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Using Numpy to store results

Numpy using int array

In [5]:
%%timeit
i = 0
results_array = np.zeros([(freq_max - freq_min + 1) * 
                    (n_banks_max - n_banks_min + 1) * 
                    (n_rows_max - n_rows_min + 1) * 
                    (n_cols_max - n_cols_min + 1) * 
                    (n_planes_max - n_planes_min + 1) * 
                    (bcm_lsb_len_max - bcm_lsb_len_min + 1), 8],
                  dtype= np.int64)
# print(results)
for freq in range(freq_min, freq_max + 1):
  for n_banks in range(n_banks_min, n_banks_max + 1):
    for n_rows in range(n_rows_min, n_rows_max + 1):
      for n_cols in range(n_cols_min, n_cols_max + 1):
        for n_planes in range(n_planes_min, n_planes_max + 1):
          for bcm_lsb_len in range(bcm_lsb_len_min, bcm_lsb_len_max + 1):
            pc = PanelConfig(
                freq = freq,
                n_banks = n_banks,
                n_rows = n_rows,
                n_cols = n_cols,
                n_planes = n_planes,
                bcm_lsb_len = bcm_lsb_len
            )
            results_array[i] = [freq, n_banks, n_rows, n_cols, n_planes, bcm_lsb_len, pc.light_efficiency * 100, pc.refresh_rate]
            i += 1

182 µs ± 811 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Numpy using float array

In [6]:
%%timeit
i = 0
results_array = np.zeros([(freq_max - freq_min + 1) * 
                    (n_banks_max - n_banks_min + 1) * 
                    (n_rows_max - n_rows_min + 1) * 
                    (n_cols_max - n_cols_min + 1) * 
                    (n_planes_max - n_planes_min + 1) * 
                    (bcm_lsb_len_max - bcm_lsb_len_min + 1), 8],
                  dtype= np.float64)
# print(results)
for freq in range(freq_min, freq_max + 1):
  for n_banks in range(n_banks_min, n_banks_max + 1):
    for n_rows in range(n_rows_min, n_rows_max + 1):
      for n_cols in range(n_cols_min, n_cols_max + 1):
        for n_planes in range(n_planes_min, n_planes_max + 1):
          for bcm_lsb_len in range(bcm_lsb_len_min, bcm_lsb_len_max + 1):
            pc = PanelConfig(
                freq = freq,
                n_banks = n_banks,
                n_rows = n_rows,
                n_cols = n_cols,
                n_planes = n_planes,
                bcm_lsb_len = bcm_lsb_len
            )
            results_array[i] = [freq, n_banks, n_rows, n_cols, n_planes, bcm_lsb_len, pc.light_efficiency * 100, pc.refresh_rate]
            i += 1

181 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Converting lists to arrays and pandas dataframes

In [7]:
results = []
i = 0
results_array = np.zeros([(freq_max - freq_min + 1) * 
                    (n_banks_max - n_banks_min + 1) * 
                    (n_rows_max - n_rows_min + 1) * 
                    (n_cols_max - n_cols_min + 1) * 
                    (n_planes_max - n_planes_min + 1) * 
                    (bcm_lsb_len_max - bcm_lsb_len_min + 1), 8],
                  dtype= np.float64)

# print(results)
for freq in range(freq_min, freq_max + 1):
  for n_banks in range(n_banks_min, n_banks_max + 1):
    for n_rows in range(n_rows_min, n_rows_max + 1):
      for n_cols in range(n_cols_min, n_cols_max + 1):
        for n_planes in range(n_planes_min, n_planes_max + 1):
          for bcm_lsb_len in range(bcm_lsb_len_min, bcm_lsb_len_max + 1):
            pc = PanelConfig(
                freq = freq,
                n_banks = n_banks,
                n_rows = n_rows,
                n_cols = n_cols,
                n_planes = n_planes,
                bcm_lsb_len = bcm_lsb_len
            )
            results.append([freq, n_banks, n_rows, n_cols, n_planes, bcm_lsb_len, pc.light_efficiency * 100, pc.refresh_rate])
            results_array[i] = [freq, n_banks, n_rows, n_cols, n_planes, bcm_lsb_len, pc.light_efficiency * 100, pc.refresh_rate]
            i += 1

List to numpy array

In [8]:
%%timeit
results_array = np.array(results)

23.1 µs ± 375 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


List to pandas DataFrame

In [9]:
%%timeit
df = pd.DataFrame(results, columns = ['freq', 'n_banks', 'n_rows', 'n_cols', 'n_planes', 'bcm_lsb_len', 'light_efficiency', 'refresh_rate'])

666 µs ± 19.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Numpy array to pandas DataFrame

In [10]:
%%timeit
df2 = pd.DataFrame(results_array, columns = ['freq', 'n_banks', 'n_rows', 'n_cols', 'n_planes', 'bcm_lsb_len', 'light_efficiency', 'refresh_rate'])

121 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Testing speed of summation equation

### Using lists to store results

In [11]:
%%timeit
results = []

# print(results)
for freq in range(freq_min, freq_max + 1):
  for n_banks in range(n_banks_min, n_banks_max + 1):
    for n_rows in range(n_rows_min, n_rows_max + 1):
      for n_cols in range(n_cols_min, n_cols_max + 1):
        for n_planes in range(n_planes_min, n_planes_max + 1):
          for bcm_lsb_len in range(bcm_lsb_len_min, bcm_lsb_len_max + 1):
            bcm = n_rows * sum((bcm_lsb_len << i) for i in range(0, n_planes))
            total = n_rows * sum( (max(bcm_lsb_len << i, n_cols) + 5) for i in range(0, n_planes))
            results.append([freq, n_banks, n_rows, n_cols, n_planes, bcm_lsb_len, bcm / total * 100, freq / total])

94.2 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
%%timeit
results = []

# print(results)
for freq in range(freq_min, freq_max + 1):
  for n_banks in range(n_banks_min, n_banks_max + 1):
    for n_rows in range(n_rows_min, n_rows_max + 1):
      for n_cols in range(n_cols_min, n_cols_max + 1):
        for n_planes in range(n_planes_min, n_planes_max + 1):
          for bcm_lsb_len in range(bcm_lsb_len_min, bcm_lsb_len_max + 1):
            bcm = n_rows * (5 * n_planes + bcm_lsb_len * (2 ** bcm_lsb_len - 1)) # Closed form of summation
            total = n_rows * sum( (max(bcm_lsb_len << i, n_cols) + 5) for i in range(0, n_planes))
            results.append([freq, n_banks, n_rows, n_cols, n_planes, bcm_lsb_len, bcm / total * 100, freq / total])

72.2 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Using Numpy to store results

Numpy using int array

In [13]:
%%timeit
i = 0
results_array = np.zeros([(freq_max - freq_min + 1) * 
                    (n_banks_max - n_banks_min + 1) * 
                    (n_rows_max - n_rows_min + 1) * 
                    (n_cols_max - n_cols_min + 1) * 
                    (n_planes_max - n_planes_min + 1) * 
                    (bcm_lsb_len_max - bcm_lsb_len_min + 1), 8],
                  dtype= np.int64)
# print(results)
for freq in range(freq_min, freq_max + 1):
  for n_banks in range(n_banks_min, n_banks_max + 1):
    for n_rows in range(n_rows_min, n_rows_max + 1):
      for n_cols in range(n_cols_min, n_cols_max + 1):
        for n_planes in range(n_planes_min, n_planes_max + 1):
          for bcm_lsb_len in range(bcm_lsb_len_min, bcm_lsb_len_max + 1):
            bcm = n_rows * sum((bcm_lsb_len << i) for i in range(0, n_planes))
            total = n_rows * sum( (max(bcm_lsb_len << i, n_cols) + 5) for i in range(0, n_planes))
            results_array[i] = [freq, n_banks, n_rows, n_cols, n_planes, bcm_lsb_len, bcm / total * 100, freq / total]
            i += 1

120 µs ± 938 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Numpy using float array

In [14]:
%%timeit
i = 0
results_array = np.zeros([(freq_max - freq_min + 1) * 
                    (n_banks_max - n_banks_min + 1) * 
                    (n_rows_max - n_rows_min + 1) * 
                    (n_cols_max - n_cols_min + 1) * 
                    (n_planes_max - n_planes_min + 1) * 
                    (bcm_lsb_len_max - bcm_lsb_len_min + 1), 8],
                  dtype= np.float64)
# print(results)
for freq in range(freq_min, freq_max + 1):
  for n_banks in range(n_banks_min, n_banks_max + 1):
    for n_rows in range(n_rows_min, n_rows_max + 1):
      for n_cols in range(n_cols_min, n_cols_max + 1):
        for n_planes in range(n_planes_min, n_planes_max + 1):
          for bcm_lsb_len in range(bcm_lsb_len_min, bcm_lsb_len_max + 1):
            bcm = n_rows * sum((bcm_lsb_len << i) for i in range(0, n_planes))
            total = n_rows * sum( (max(bcm_lsb_len << i, n_cols) + 5) for i in range(0, n_planes))
            results_array[i] = [freq, n_banks, n_rows, n_cols, n_planes, bcm_lsb_len, bcm / total * 100, freq / total]
            i += 1

120 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Conclusion

The fastest method is to create the data using the summation equation and then convert to whatever format is most convenient after that. List -> Numpy Array -> Pandas DataFrame seems to be the fastest path but when you convert it to Numpy Array everything becomes the same type so the best way is to go from List -> Pandas DataFrame